# Traffic Sign Classifier - Inference Application

This notebook loads the trained GTSRB traffic sign classifier model and performs inference on:
- **Images** stored in `input/` folder
- **Videos** (optional) - extracts frames and classifies traffic signs

**Model Used:** `GTSRB_resnet50_E18_VAL100.00.pth`  
**Classes:** 43 German Traffic Sign classes

---

## Instructions

1. **Place your images or videos** in the `input/` folder
2. **Run all cells** in order (Cell → Run All)
3. **View results** with predicted classes and confidence scores

**Supported Formats:**
- Images: `.png`, `.jpg`, `.jpeg`, `.bmp`, `.tiff`, `.webp`
- Videos: `.mp4`, `.avi`, `.mov`, `.mkv`, `.wmv`

### 2. Configuration and Device Setup

In [19]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


### 1. Import Libraries

In [20]:
# =======================================
# Configuration
# =======================================
MODEL_PATH = "model/GTSRB_resnet50_E18_VAL100.00.pth"
INPUT_FOLDER = "input"
NUM_CLASSES = 43  # GTSRB has 43 traffic sign classes
IMG_SIZE = 224

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

✅ Using device: cuda


In [21]:
# Process videos in input folder (optional)
valid_video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.wmv')

video_files = [f for f in os.listdir(INPUT_FOLDER) 
               if f.lower().endswith(valid_video_extensions)]

if video_files:
    print(f"\n{'='*80}")
    print(f"🎬 Found {len(video_files)} video(s) in input folder")
    print(f"{'='*80}\n")
    
    for video_filename in video_files:
        video_path = os.path.join(INPUT_FOLDER, video_filename)
        print(f"📹 Processing: {video_filename}")
        print(f"{'─'*80}")
        
        try:
            video_results = process_video(video_path, model, preprocess, device, class_names, frame_interval=30)
            print(f"{'─'*80}\n")
        except Exception as e:
            print(f"❌ Error processing video {video_filename}: {str(e)}\n")
else:
    print(f"\n💡 No videos found in input folder.")
    print(f"   To process videos, add .mp4, .avi, or other video files to: {INPUT_FOLDER}/")


💡 No videos found in input folder.
   To process videos, add .mp4, .avi, or other video files to: input/


### 8. Video Processing Function (Optional)

In [22]:
def process_video(video_path, model, preprocess, device, class_names, frame_interval=30):
    """
    Process video and classify traffic signs in sampled frames.
    
    Args:
        video_path: Path to input video
        model: Trained PyTorch model
        preprocess: Preprocessing transforms
        device: torch device (cuda/cpu)
        class_names: Dictionary mapping class IDs to names
        frame_interval: Process every Nth frame (default: 30)
        
    Returns:
        List of predictions for sampled frames
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Error: Cannot open video {video_path}")
        return []
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"📹 Video Info:")
    print(f"   FPS: {fps:.2f}")
    print(f"   Total Frames: {total_frames}")
    print(f"   Processing every {frame_interval} frames\n")
    
    results = []
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process every Nth frame
        if frame_count % frame_interval == 0:
            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            
            # Preprocess and predict
            img_tensor = preprocess(img).unsqueeze(0).to(device)
            
            with torch.no_grad():
                outputs = model(img_tensor)
                probabilities = torch.softmax(outputs, dim=1)
                confidence, predicted_class = torch.max(probabilities, 1)
            
            predicted_id = predicted_class.item()
            predicted_label = class_names.get(predicted_id, f"Unknown ({predicted_id})")
            conf_percentage = confidence.item() * 100
            
            timestamp = frame_count / fps
            
            result = {
                "frame": frame_count,
                "timestamp": f"{timestamp:.2f}s",
                "predicted_class_id": predicted_id,
                "predicted_class_name": predicted_label,
                "confidence": conf_percentage
            }
            results.append(result)
            
            print(f"⏱️  Frame {frame_count} ({timestamp:.2f}s): {predicted_label} ({conf_percentage:.2f}%)")
        
        frame_count += 1
    
    cap.release()
    print(f"\n✅ Processed {len(results)} frames from video")
    
    return results

print("✅ Video processing function defined")

✅ Video processing function defined


### 9. Process Videos in Input Folder (Optional)

### 7. Process All Images in Input Folder

In [23]:
# Process all images in input folder
valid_image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')

print(f"\n{'='*80}")
print(f"🔍 Scanning input folder: {INPUT_FOLDER}")
print(f"{'='*80}\n")

image_files = [f for f in os.listdir(INPUT_FOLDER) 
               if f.lower().endswith(valid_image_extensions)]

if not image_files:
    print("⚠️  No images found in input folder!")
    print(f"   Please add images to: {INPUT_FOLDER}/")
    print(f"   Supported formats: {', '.join(valid_image_extensions)}")
else:
    print(f"📁 Found {len(image_files)} image(s)\n")
    
    results = []
    for filename in image_files:
        img_path = os.path.join(INPUT_FOLDER, filename)
        
        try:
            result = predict_image(img_path, model, preprocess, device, class_names)
            results.append(result)
            
            print(f"📷 {result['file']}")
            print(f"   🏷️  Predicted: {result['predicted_class_name']}")
            print(f"   🎯 Confidence: {result['confidence']:.2f}%")
            print(f"   {'─'*60}")
            
        except Exception as e:
            print(f"❌ Error processing {filename}: {str(e)}")
            print(f"   {'─'*60}")
    
    print(f"\n{'='*80}")
    print(f"✅ Processed {len(results)}/{len(image_files)} images successfully")
    print(f"{'='*80}")


🔍 Scanning input folder: input

📁 Found 1 image(s)

📷 images.png
   🏷️  Predicted: Class 1
   🎯 Confidence: 21.34%
   ────────────────────────────────────────────────────────────

✅ Processed 1/1 images successfully


### 6. Inference Function for Images

In [24]:
def predict_image(image_path, model, preprocess, device, class_names):
    """
    Predict traffic sign class for a single image.
    
    Args:
        image_path: Path to input image
        model: Trained PyTorch model
        preprocess: Preprocessing transforms
        device: torch device (cuda/cpu)
        class_names: Dictionary mapping class IDs to names
        
    Returns:
        Dictionary with prediction results
    """
    # Load and preprocess image
    img = Image.open(image_path).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(img_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted_class = torch.max(probabilities, 1)
    
    predicted_id = predicted_class.item()
    predicted_label = class_names.get(predicted_id, f"Unknown ({predicted_id})")
    conf_percentage = confidence.item() * 100
    
    return {
        "file": os.path.basename(image_path),
        "predicted_class_id": predicted_id,
        "predicted_class_name": predicted_label,
        "confidence": conf_percentage
    }

print("✅ Image inference function defined")

✅ Image inference function defined


### 5. Define Preprocessing Pipeline

In [25]:
# Image preprocessing (same as training)
preprocess = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ Preprocessing pipeline defined")

✅ Preprocessing pipeline defined


### 4. Load Class Names

In [26]:
# Load class names from Meta.csv
meta_path = "../dataset/Meta.csv"
if os.path.exists(meta_path):
    meta_df = pd.read_csv(meta_path)
    class_names = {row['ClassId']: f"Class {row['ClassId']}" for _, row in meta_df.iterrows()}
    print(f"✅ Loaded {len(class_names)} class names from Meta.csv")
else:
    # Fallback: use generic class names
    class_names = {i: f"Class {i}" for i in range(NUM_CLASSES)}
    print(f"⚠️  Meta.csv not found, using generic class names")

✅ Loaded 43 class names from Meta.csv


### 3. Load Trained Model

In [27]:
# Load the trained ResNet50 model
print("🔄 Loading model...")

# Recreate model architecture
model = models.resnet50(weights=None)
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(in_features, NUM_CLASSES)
)

# Load trained weights
checkpoint = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

print(f"✅ Model loaded successfully!")
print(f"   Architecture: ResNet50")
print(f"   Classes: {NUM_CLASSES}")
print(f"   Best Epoch: {checkpoint['epoch']}")
print(f"   Validation Accuracy: {checkpoint['val_acc']*100:.2f}%")

🔄 Loading model...
✅ Model loaded successfully!
   Architecture: ResNet50
   Classes: 43
   Best Epoch: 18
   Validation Accuracy: 100.00%
✅ Model loaded successfully!
   Architecture: ResNet50
   Classes: 43
   Best Epoch: 18
   Validation Accuracy: 100.00%
